In [1]:
%matplotlib qt

import numpy as np
import scipy.linalg as linalg
from matplotlib.pyplot import *
import pickle
from quadcopter.util import *

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 160
rcParams['figure.autolayout'] = True
rcParams['pgf.rcfonts'] = False
rcParams['lines.linewidth'] = 1.2


colors = {
    'reference': 'black',
    'ekf': 'red',
    'mhe1': ['green', 'olive', 'jungle green'],
    'mhe2': ['blue', 'amethyst', 'water blue']
}


def save_plot(plot_name):
    savefig(f"C:\\Users\\Robert\\Seafile\\Uni\\Studienarbeit\\Arbeit\\plots\\{plot_name}.pgf")


def to_deg(rad):
    return rad / np.pi * 180.0


def load_results(file_name):
    complete_path = f"C:\\dev\\mhe_pymoskito\\quadcopter\\results\\simulation\\{file_name}.pmr"
    with open(complete_path, "rb") as f:
        return pickle.load(f)['results']
    

class ExperimentData:
    def __init__(self, ex_id, raw):
        self.ex_id = ex_id
        self.raw = raw
    
    @property
    def time(self):
        return self.raw['time']
    
    @property
    def ref_pos(self):
        return self.raw['Observer'][:, :3].T
               
    @property
    def ref_ori(self):
        return self.raw['Observer'][:, 3:6].T
    
    @property
    def obs_ori(self):
        return self.raw['Observer'][:, 6:9].T
               
    @property
    def obs_error(self):
        return self.raw['Observer'][:, 9:12].T
    
    @property
    def obs_gyro_bias(self):
        return self.raw['Observer'][:, 12:15].T
    
    @property
    def obs_acc_bias(self):
        return self.raw['Observer'][:, 15:18].T
    
    @property
    def meas_gyro(self):
        return self.raw['Observer'][:, 18:21].T
    
    @property
    def meas_acc(self):
        return self.raw['Observer'][:, 21:24].T
    
    
def load_experiment(ex_id, resample_steps=None, start_time=None, end_time=None) -> ExperimentData:
    raw = load_results(ex_id)
    
    start_index = None
    end_index = None
    
    if start_time is not None:
        start_index = time_to_index(start_time, raw['time'])
        
    if end_time is not None:
        end_index = time_to_index(end_time, raw['time'])
    
    raw['time'] = raw['time'][start_index:end_index:resample_steps]
    raw['Solver'] = raw['Solver'][start_index:end_index:resample_steps]
    raw['Observer'] = raw['Observer'][start_index:end_index:resample_steps]
    raw['Model'] = raw['Model'][start_index:end_index:resample_steps]
    return ExperimentData(ex_id, raw)

In [3]:
uav3_ekf_with_error = load_experiment('uav3_ekf_with_error')
uav3_mhe20_with_error = load_experiment('uav3_mhe20_with_error')

room3_ekf_with_error = load_experiment('room3_ekf_with_error')
room3_mhe20_with_error = load_experiment('room3_mhe20_with_error')

experiments = [('uav3', uav3_ekf_with_error, uav3_mhe20_with_error),
               ('room3', room3_ekf_with_error, room3_mhe20_with_error)]

for ex in experiments:
    ex_name = ex[0]
    ekf_data = ex[1]
    mhe_data = ex[2]
    
    figure(figsize=(6.5, 4))
    subplot(221)
    ref_line, = plot(ekf_data.time, ekf_data.ref_ori[0], color=colors['reference'])
    ekf_line, = plot(ekf_data.time, ekf_data.obs_ori[0], color=colors['ekf'], linestyle='--')
    mhe_line, = plot(mhe_data.time, mhe_data.obs_ori[0], color=colors['mhe1'][0], linestyle='--')
    xlabel(r"$t$ (s)")
    ylabel(r"$\phi$ (deg)")
    grid()
    
    subplot(222)
    plot(ekf_data.time, ekf_data.obs_error[0], color=colors['ekf'], linestyle='--')
    plot(mhe_data.time, mhe_data.obs_error[0], color=colors['mhe1'][0], linestyle='--')
    xlabel(r"$t$ (s)")
    ylabel(r"$\mathit{\Delta \phi}$ (deg)")
    grid()
    
    subplot(223)
    plot(ekf_data.time, ekf_data.ref_ori[1], color=colors['reference'])
    plot(ekf_data.time, ekf_data.obs_ori[1], color=colors['ekf'], linestyle='--')
    plot(mhe_data.time, mhe_data.obs_ori[1], color=colors['mhe1'][0], linestyle='--')
    xlabel(r"$t$ (s)")
    ylabel(r"$\theta$ (deg)")
    grid()
    
    subplot(224)
    plot(ekf_data.time, ekf_data.obs_error[1], color=colors['ekf'], linestyle='--')
    plot(mhe_data.time, mhe_data.obs_error[1], color=colors['mhe1'][0], linestyle='--')
    xlabel(r"$t$ (s)")
    ylabel(r"$\mathit{\Delta \theta}$ (deg)")
    grid()
    
    figlegend((ref_line, ekf_line, mhe_line),
              ('Referenz', 'EKF', 'MHE N=20'))
    
    tight_layout()
    save_plot(f"{ex_name}_roll_pitch")
    show()
    
    figure(figsize=(4, 3))
    ekf_line, = plot(ekf_data.time, linalg.norm(ekf_data.obs_error, axis=0), color=colors['ekf'])
    mhe_line, = plot(mhe_data.time, linalg.norm(mhe_data.obs_error, axis=0), color=colors['mhe1'][0])
    xlabel(r"$t$ (s)")
    ylabel(r"Fehlernorm Eulerwinkel (deg)")
    grid()
    
    figlegend((ekf_line, mhe_line),
              ('EKF', 'MHE N=20'))
    
    save_plot(f"{ex_name}_error_norm")
    show()

C:\dev\Python\studienarbeit\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['lmodern'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
